In [ ]:
import os
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
from __future__ import absolute_import, division, print_function

import numpy as np
from matplotlib import pyplot as plt
from matplotlib.patches import Patch

from sklearn.model_selection import train_test_split

import tensorflow as tf
import keras
import keras.backend as K
from keras.layers import Dense, Input
from keras.models import Model
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [ ]:
import time
import ROOT
import numpy as np
import pandas as pd
import root_pandas as rpd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.colors import LogNorm
from root_pandas import read_root
from matplotlib import rc
from numpy import inf


import sys,pandas as pd, matplotlib , matplotlib.pyplot as plt, matplotlib.lines , numpy as np, math, pylab
#import ROOT
#from ROOT import TFile

import root_pandas
%matplotlib inline

rc('text', usetex=True)

import matplotlib as mpl
rc('font', family='serif')
rc('text', usetex=True)
rc('font', size=22)
rc('xtick', labelsize=15)
rc('ytick', labelsize=15)
rc('legend', fontsize=15)

#
mpl.rcParams.update({'font.size': 19})
#mpl.rcParams.update({'legend.fontsize': 18})
mpl.rcParams.update({'xtick.labelsize': 18}) 
mpl.rcParams.update({'ytick.labelsize': 18}) 
mpl.rcParams.update({'text.usetex' : False})
mpl.rcParams.update({'axes.labelsize': 18}) 
mpl.rcParams.update({'legend.frameon': False}) 

In [ ]:
# Define default plot styles
plot_style_0 = {
    'histtype': 'step',
    'color': 'black',
    'linewidth': 2,
    'linestyle': '--',
    'density': True
}

plot_style_1 = {
    'histtype': 'step',
    'color': 'black',
    'linewidth': 2,
    'density': True
}

plot_style_2 = {'alpha': 0.5, 'density': True}

In [ ]:
# Check Versions
print(tf.__version__)  # 1.15.0
print(keras.__version__)  # 2.2.4

## Get dataset and MC

In [ ]:
path = '/home/miguel/EG2_DATA/sebastian/'
data_electrons = root_pandas.read_root(path+'C.root', "ntuple_data_electrons")
data_hadron = root_pandas.read_root(path+'C.root', "ntuple_data")

In [ ]:
data_electrons = data_electrons.query('TargType==2')
data_hadron    = data_hadron.query('TargType==2')

In [ ]:
path = '/home/miguel/EG2_DATA/andres/'
mc_electrons = root_pandas.read_root(path+'MC_C.root', "ntuple_e")
mc_hadron = root_pandas.read_root(path+'MC_C.root', "ntuple_sim")
mc_hadron['PID'] = mc_hadron['pid']
mc_hadron['mc_PID'] = mc_hadron['mc_pid']

mc_hadron['pass_reco'] = np.where(mc_hadron['Pt2']>0, True, False)
mc_hadron['pass_truth'] = np.where(mc_hadron['mc_Pt2']>0, True, False)

data_hadron['pass_reco'] = np.where(data_hadron['Pt2']>0, True, False)





In [ ]:
data = data_hadron.sample(n=500000)
mc   = mc_hadron.sample(n=500000)

In [ ]:

##Hadron momentum
Px_0_G = mc['mc_Px']
Px_0_S =mc['Px']
Px_unknown_S = data['Px']
Px_unknown_G = data['Px']
Px_0_G[mc['pass_truth']==False] = -10
Px_0_S[mc['pass_reco']==False] = -10

Py_0_G = mc['mc_Py']
Py_0_S =mc['Py']
Py_unknown_S = data['Py']
Py_unknown_G = data['Py']
Py_0_G[mc['pass_truth']==False] = -10
Py_0_S[mc['pass_reco']==False] = -10

Pz_0_G = mc['mc_Pz']
Pz_0_S =mc['Pz']
Pz_unknown_S = data['Pz']
Pz_unknown_G = data['Pz']
Pz_0_G[mc['pass_truth']==False] = -10
Pz_0_S[mc['pass_reco']==False] = -10

Pid_0_G = mc['mc_PID']
Pid_0_S =mc['PID']
Pid_unknown_S = data['PID']
Pid_unknown_G = data['PID']
Pid_0_G[mc['pass_truth']==False] = -10
Pid_0_S[mc['pass_reco']==False] = -10



Q2_0_G = mc['mc_Q2']
Q2_0_S =mc['Q2']
Q2_unknown_S = data['Q2']
Q2_unknown_G = data['Q2']
Q2_0_G[mc['pass_truth']==False] = -10
Q2_0_S[mc['pass_reco']==False] = -10





In [ ]:
theta0_G=(Px_0_G)
theta0_S=(Px_0_S)
theta_unknown_S=(Px_unknown_S)
theta_unknown_G=(Px_unknown_G)

In [ ]:
theta0_G=(Px_0_G,Px_0_G)
theta0_S=(Px_0_S,Px_0_S)
theta_unknown_S=(Px_unknown_S,Px_unknown_S)
theta_unknown_G=(Px_unknown_G,Px_unknown_G)

In [ ]:
theta0 = np.stack([theta0_G, theta0_S], axis=1)
labels0 = np.zeros(len(theta0))
theta_unknown = np.stack([theta_unknown_G, theta_unknown_S], axis=1)
labels_unknown = np.ones(len(theta_unknown))


xvals_1 = np.concatenate((theta0_S, theta_unknown_S))
yvals_1 = np.concatenate((labels0, labels_unknown))

xvals_2 = np.concatenate((theta0_G, theta0_G))
yvals_2 = np.concatenate((labels0, labels_unknown))

In [ ]:
print(len(xvals_1),len(yvals_1))

In [ ]:
print(xvals_1)

In [ ]:
print(yvals_1)

## The following one fails when I give a list of 2 numpy arrays as theta0_G etc. It works fine if I give just a list of 1 element. 

In [ ]:
xvals_1[xvals_1[:,0]!=-10] 
yvals_1[xvals_1[:,0]!=-10]

In [ ]:
len(Py_0_G)

In [ ]:
theta0_G=(Px_0_G,Py_0_G,Pz_0_G,Pid_0_G,Q2_0_G)
theta0_S=(Px_0_S,Py_0_S,Pz_0_S,Pid_0_S,Q2_0_S)
theta_unknown_S=(Px_unknown_G,Py_unknown_G,Pz_unknown_G,Pid_unknown_G,Q2_unknown_G)
theta_unknown_G=(Px_unknown_G,Py_unknown_G,Pz_unknown_G,Pid_unknown_G,Q2_unknown_G)


# OmniFold Gaussian Toy Example

## Setup the Datasets

In [ ]:
theta0 = np.stack([theta0_G, theta0_S], axis=1)
labels0 = np.zeros(len(theta0))


theta_unknown = np.stack([theta_unknown_G, theta_unknown_S], axis=1)
labels_unknown = np.ones(len(theta_unknown))

In [ ]:
bins = np.linspace(0, 11, 41)

fig, ax = plt.subplots(1,
                       2,
                       figsize=(12, 5),
                       constrained_layout=True,
                       sharey=True)

ax[0].set_xlabel('$x$')
ax[0].set_ylabel('Events per bin (normalized)')
ax[0].hist(theta0_G[theta0_G!=-10], bins=bins, **plot_style_2, label='Generation')
ax[0].hist(theta0_S[theta0_S!=-10], bins=bins, **plot_style_2, label='Simulation')
legend = ax[0].legend(
    loc='upper left',
    frameon=True)
plt.setp(legend.get_title(), multialignment='center')

ax[1].set_xlabel('$x$')
ax[1].hist(theta_unknown_G[theta_unknown_G!=-10], bins=bins, **plot_style_2, label='Truth')
ax[1].hist(theta_unknown_S[theta_unknown_S!=-10], bins=bins, **plot_style_2, label='Detector')
legend = ax[1].legend(
    loc='upper left',
    frameon=True)
plt.setp(legend.get_title(), multialignment='center')

fig.show()

## Unfold Data

In [ ]:
iterations = 1

In [ ]:
xvals_1 = np.concatenate((theta0_S, theta_unknown_S))
yvals_1 = np.concatenate((labels0, labels_unknown))

xvals_2 = np.concatenate((theta0_G, theta0_G))
yvals_2 = np.concatenate((labels0, labels_unknown))

weights = np.empty(shape=(iterations, 2, len(theta0)))
# shape = (iteration, step, event)

inputs = Input((1, ))
hidden_layer_1 = Dense(50, activation='relu')(inputs)
hidden_layer_2 = Dense(50, activation='relu')(hidden_layer_1)
hidden_layer_3 = Dense(50, activation='relu')(hidden_layer_2)
outputs = Dense(1, activation='sigmoid')(hidden_layer_3)

model = Model(inputs=inputs, outputs=outputs)

earlystopping = EarlyStopping(patience=10,
                              verbose=1,
                              restore_best_weights=True)

In [ ]:
# from NN (DCTR)
def reweight(events):
    f = model.predict(events, batch_size=10000)
    weights = f / (1. - f)
    return np.squeeze(np.nan_to_num(weights))

In [ ]:
# initial iterative weights are ones

weights_pull = np.ones(len(theta0_S))
weights_push = np.ones(len(theta0_S))

In [ ]:
K.clear_session()


In [ ]:
for i in range(iterations):
    print("\nITERATION: {}\n".format(i + 1))

    # STEP 1: classify Sim. (which is reweighted by weights_push) to Data
    # weights reweighted Sim. --> Data
    print("STEP 1\n")

    weights_1 = np.concatenate((weights_push, np.ones(len(theta_unknown_S))))
    # actual weights for Sim., ones for Data (not MC weights)

    X_train_1, X_test_1, Y_train_1, Y_test_1, w_train_1, w_test_1 = train_test_split(
        xvals_1, yvals_1, weights_1)

    model.compile(loss='binary_crossentropy',
                  optimizer='Adam',
                  metrics=['accuracy'])
    model.fit(X_train_1[X_train_1[:,0]!=-10],
              Y_train_1[X_train_1[:,0]!=-10],
              sample_weight=w_train_1[X_train_1[:,0]!=-10],
              epochs=200,
              batch_size=10000,
              validation_data=(X_test_1[X_test_1[:,0]!=-10], Y_test_1[X_test_1[:,0]!=-10], w_test_1[X_test_1[:,0]!=-10]),
              callbacks=[earlystopping],
              verbose=1)

    weights_pull = weights_push * reweight(theta0_S)
    weights_pull[theta0_S==-10] = 1. #these are events that don't pass reco; take the prior.
    weights[i, :1, :] = weights_pull

    # STEP 2: classify Gen. to reweighted Gen. (which is reweighted by weights_pull)
    # weights Gen. --> reweighted Gen.
    print("\nSTEP 2\n")

    weights_2 = np.concatenate((np.ones(len(theta0_G)), weights_pull))
    # ones for Gen. (not MC weights), actual weights for (reweighted) Gen.

    X_train_2, X_test_2, Y_train_2, Y_test_2, w_train_2, w_test_2 = train_test_split(
        xvals_2, yvals_2, weights_2)

    model.compile(loss='binary_crossentropy',
                  optimizer='Adam',
                  metrics=['accuracy'])
    model.fit(X_train_2,
              Y_train_2,
              sample_weight=w_train_2,
              epochs=200,
              batch_size=10000,
              validation_data=(X_test_2, Y_test_2, w_test_2),
              callbacks=[earlystopping],
              verbose=1)

    weights_push = reweight(theta0_G)
    weights[i, 1:2, :] = weights_push

In [ ]:
bins = np.linspace(-8, 8, 41)

for i in range(2):
    print("ITERATION {}:".format(i + 1))
    plt.figure(figsize=(6, 5))
    hist0 = plt.hist(theta0_G[theta0_G!=0],
                     bins=bins,
                     **plot_style_2)

    hist1 = plt.hist(
        theta0_G[theta0_G!=0],
        bins=bins,
        weights=weights[i, 1, :][theta0_G!=0],
        **plot_style_1)
    hist2 = plt.hist(theta_unknown_G[theta_unknown_G!=0],
                     bins=bins,
                     label=r'($\mu$, $\sigma$) = ?',
                     **plot_style_2)
    plt.legend(frameon=False)
    plt.xlabel(r"$x_G$")
    plt.ylabel("Events per bin (normalized)")
    plt.show()

In [ ]:
P_0_S

In [ ]:
for i in range(len(weights)):

    print("ITERATION: {}".format(i + 1))
    bins = np.linspace(1.0, 4.0, 40)
    
    fig, ax = plt.subplots(3,
                           3,
                           figsize=(16, 16),
                           constrained_layout=True)
    ax[0,0].set_xlabel('P')
    ax[0,0].set_ylabel('Events per bin (Normalized)')
    ax[0,0].hist(P_0_S, bins=bins, label=r'MC .', **plot_style_2)
    ax[0,0].hist(P_0_S,
               bins=bins,
               weights=weights[i, 0, :],
               label=r'MC . DCTR wgt.',
               **plot_style_1)

    ax[0,0].hist(P_unknown_S,
               bins=bins,
               label=r'Data (reco)',
               **plot_style_2)
    ax[0,0].legend(frameon=False,loc='best')